<h1 style="text-align: center;">User Expenses Prediction - Data Preparation</h1>

In [1]:
import pandas as pd 
import numpy as np

In [2]:
# Sheet has two tabs so we import both tabs
df1 = pd.read_excel('dataset.xlsx', sheet_name=0)  # First sheet
df2 = pd.read_excel('dataset.xlsx', sheet_name=1)  # Second sheet

In [3]:
mode_df1 = df1['Customer ID'].value_counts().to_dict()
top_5_df1 = dict(list(mode_df1.items())[:5])
print(top_5_df1)

{14911.0: 5710, 17841.0: 5114, 14606.0: 3927, 14156.0: 2710, 12748.0: 2665}


In [4]:
mode_df2 = df2['Customer ID'].value_counts().to_dict()
top_5_df2 = dict(list(mode_df2.items())[:5])
print(top_5_df2)

{17841.0: 7983, 14911.0: 5903, 14096.0: 5128, 12748.0: 4642, 14606.0: 2782}


We have two users that have high records in both dataset (i.e 2009-2011):
1. User ID <b>14911</b>: 5710 + 5903 = 11613
2. User ID <b>17841</b>: 5114 + 7983 = 13097

These two users have highest number of records in both dataset. These two users can be taken as reference for our dataset preparation. Between those two we need to find the best fit user with less inconsistency in the dataset.

Testing for user 14911 (user_x)

In [ ]:
user_x = df1[df1['Customer ID'] == 14911]
user_x.set_index('InvoiceDate').sort_index()

440       EIRE
441       EIRE
442       EIRE
443       EIRE
444       EIRE
          ... 
523021    EIRE
523022    EIRE
523023    EIRE
523024    EIRE
523025    EIRE
Name: Country, Length: 5710, dtype: object